In [73]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.contrib import rnn
import timeit
import tflearn
from tflearn.layers.recurrent import lstm
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.estimator import regression
from tflearn.layers.core import input_data, dropout, fully_connected, time_distributed, flatten, activation
# import tensorflow.rnn as rnn
# from tensorflow.data import DataSet 

In [74]:
tf.reset_default_graph()

In [75]:
epoch = 20

batch_size = 32

#Prepare input data
classes = ['Abort','Circle', 'Hello', 'No', 'Stop', 'Turn Left', 'Turn Right', 
         'Turn', 'Warn', 'No_motion'] #['Circle','Turn Left', 'Turn Right']#
num_classes = len(classes)

In [76]:
# define example
# data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = np.array(classes)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

['Abort' 'Circle' 'Hello' 'No' 'Stop' 'Turn Left' 'Turn Right' 'Turn'
 'Warn' 'No_motion']
[0 1 2 3 5 7 8 6 9 4]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]
['Abort']


In [77]:
[np.argmax(onehot_encoded[0, :])]

[0]

In [78]:
# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = (64, 48)
num_channels = 1
train_path='training_data'

In [79]:
x_data = []
y_label = []
path = r'C:\\Study\\Sem 3\\ChrisTseng\\gesture\\Save1'
for g in os.listdir(path):
    print (g)
    gp = os.path.join(path,g)
    maxi = 100
    for s in os.listdir(gp): #sample 1_0
        sp = os.path.join(gp, s)
        t = [] #np.array([], np.int32)
        for i in os.listdir(sp):
            a = np.asarray(Image.open(os.path.join(sp, i)))
            if a.shape != (48,64):
                print(sp+" : "+str(a.shape))
            if len(t) != 22:
                t.append(np.transpose(a/255.0))
            else:
                break
#             maxi = max(maxi,i)
        t = np.pad(t, ((0,22-len(t)),(0,0),(0,0)), 'constant', constant_values=[0])
        x_data.append(np.array(t))
        y_label.append(g)
        maxi = min(len(t), maxi)
    print("maxi: "+str(maxi))
#     img = cv2.imread(os.path.join(path, f))

abort
maxi: 22
circle
maxi: 22
hello
maxi: 22
no
maxi: 22
stop
maxi: 22
turn
maxi: 22
turn_left
maxi: 22
turn_right
maxi: 22
warn
maxi: 22


In [80]:
#Adding samples of black differential image as 'No_motion'
for i in range(60):
    x_data.append(np.zeros((22,64,48), int))
    y_label.append('No_motion')

In [81]:
# x_data = x_data/255.0

In [82]:
y_enc_label = onehot_encoder.fit_transform(label_encoder.fit_transform(y_label).reshape(len(y_label), 1))

In [83]:
x_data_a = np.array(x_data)

In [84]:
len(x_data)

603

In [85]:
x_data[5].shape

(22, 64, 48)

In [86]:
x_data_a.shape

(603, 22, 64, 48)

In [87]:
len(x_data_a[4])


22

In [88]:
y_enc_label.shape

(603, 10)

In [89]:
X = x_data_a.reshape(-1, 22, 64, 48, 1)
y = y_enc_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [90]:
# data_sets = 
# data_sets.train = DataSet(X_train, y_train)
# data_sets.validation = DataSet(X_val, y_val)
# data_sets.test = DataSet(X_test, y_test)
X_train.shape

(385, 22, 64, 48, 1)

In [91]:
x = tf.placeholder(tf.float32, shape=[None, img_size[0], img_size[1], num_channels], name='x')

In [92]:
## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [93]:
##Network graph params
no_of_frames = 22
filter_size_conv1 = 11
num_filters_conv1 = 5

filter_size_conv2 = 6
num_filters_conv2 = 10

filter_size_conv3 = 5
num_filters_conv3 = 5

filter_size_conv4 = 2
num_filters_conv4 = 2

lstm_units = 500
    
fc_layer_size = 500

learning_rate = 1e-4

In [94]:
# net = tflearn.input_data([None, 22, 64, 48], name="input")
# net = time_distributed(net, conv_2d, args=[nb_filter=num_filters_conv1,
#                                   filter_size=filter_size_conv1,
#                                   activation='tanh'])
# net = time_distributed(net, max_pool_2d, args=[kernel_size=2])
# net = time_distributed(net, conv_2d, args=[nb_filter=num_filters_conv2,
#                                   filter_size=filter_size_conv2,
#                                   activation='tanh'])
# net = time_distributed(net, max_pool_2d, args=[kernel_size=2])
# net = time_distributed(net, flatten, args=[name='flat'])

In [95]:
tflearn.config.init_training_mode()
net = tflearn.input_data([None, 22, 64, 48, 1], name="input")
#layer_1 CNN
net = conv_3d(net, num_filters_conv1, filter_size_conv1, 1, 'same', 'tanh', True, 'xavier')
net = max_pool_3d(net, 2)
#dropout
net = dropout(net, 0.8)
#layer_2 CNN
net = conv_3d(net, num_filters_conv2,filter_size_conv2, 1, 'same', 'tanh', True, 'xavier')
net = max_pool_3d(net, 2)

#layer_3 CNN
net = conv_3d(net, num_filters_conv3, filter_size_conv3, 1, 'same', 'tanh', True, 'xavier')
net = max_pool_3d(net, 2)
# #layer_4 CNN
# net = time_distributed(net, conv_2d, args=[num_filters_conv4,
#                                   filter_size_conv4,1,'same',
#                                   'tanh', True, 'xavier'])
# net = time_distributed(net, max_pool_2d, args=[2])
#dropout
net = dropout(net, 0.8)
net = time_distributed(net, flatten, args=['flat'])

In [96]:
# net = time_distributed(net, dense, args=[fc_layer_size])

In [97]:
net

<tf.Tensor 'concat:0' shape=(?, 22, 15360) dtype=float32>

In [98]:
net = lstm(net, lstm_units, dropout=(0.9,0.7))

In [99]:
net

<tf.Tensor 'LSTM/LSTM/cond_43/Merge:0' shape=(?, 500) dtype=float32>

In [100]:
fc_layer = tflearn.fully_connected(net, num_classes, activation='softmax')

In [101]:
fc_layer

<tf.Tensor 'FullyConnected/Softmax:0' shape=(?, 10) dtype=float32>

In [102]:
loss = tflearn.objectives.categorical_crossentropy (fc_layer, y_true)

In [103]:
loss

<tf.Tensor 'Crossentropy/Mean:0' shape=() dtype=float32>

In [104]:
network = regression(fc_layer, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Training
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='gestureCNNLSTM.tfl.ckpt',
                    best_checkpoint_path='./best_gestureCNNLSTM', max_checkpoints=2)

In [105]:
model.fit(X_train, y_train, validation_set=(X_val, y_val), n_epoch=epoch, shuffle=True,
          show_metric=True, batch_size=32, run_id='gesture17')

Training Step: 3  | total loss: 2.17879 | time: 1239.568s
| Adam | epoch: 001 | loss: 2.17879 - acc: 0.3068 -- iter: 096/385


KeyboardInterrupt: 

In [ ]:
model.save("gestureCNNLSTM.tfl")

In [ ]:
X_train.shape